## Liste des librairies utilisées

In [2]:
import numpy as np 
import pandas as pd 
import os
import pydicom
import matplotlib.pyplot as plt
from tqdm import tqdm

## Fichier "sample_submission"

In [3]:
sample = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
print(len(sample)) #87 individus dans l'échantillon test
sample.head(10) #Visualise les 10 premières lignes

Commentaire : BraTS21ID : Correspond à l'identifiant. Permet de faire le lien avec les images
MGMT_value : proba de la présence de MGMT. 1 = Présence. Détecter le MGMT permet de prévenir la tumeur du cerveau

## Base train

In [4]:
train = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
print(len(train))#585 individus dans le fichier train
train.head(10)#Visualise les 10 premières lignes

## Visualisation d'une image

In [5]:
dataset = pydicom.filereader.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-109.dcm') #Importation d'une seule image pour visualiser
img = dataset.pixel_array #La fonction permet de lire les pixel dans une forme plus pratique
fig, ax = plt.subplots()#Création d'un plot
ax.imshow(img, cmap='Greys')#Charge l'image en nuance de gris
ax.set_axis_off()# Pas de visualisation des axes car il s'agit d'une image
plt.show() #Affiche l'image

## Etude des images

In [6]:
train["imfolder"] = ['{0:05d}'.format(s) for s in train["BraTS21ID"]]
#0:05d --> Permet d'ajouter quatre "0"
#L'objectif de cette commande est de relier chaque identifiant à un identifiant de fichier qui contient les images associées à l'identifiant
train.head(10)

In [7]:
#On définit le chemin d'accès aux images associées à chaque id
train_path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train" #base du chemin, commune à tous les id
train["path"] = [os.path.join(train_path,s) for s in train["imfolder"]  ]# Ajout du numéro de dossier définit précédement
train.head(10)#On visualise la colonne qui contient le chemin vers les images

In [8]:
#Défini une liste :
Scans = ["FLAIR","T1w","T1wCE","T2w"]
#Utile pour la suite, pour la navigation au sein des fichiers avec une boucle

Calcul du nombre d'images par dossier pour le premier id

In [9]:
for type_img in Scans :
    print(type_img , " : ", len(os.listdir(os.path.join(train["path"].iloc[0],type_img))), " images")


In [10]:
for scan in Scans: #Compte le nombre de sous-dossiers parmi les dossiers définit dans "Scans" 
    train[scan +"_count"] = [ len(os.listdir(os.path.join(train["path"].iloc[s],scan))) for s in tqdm(range(len(train))) ]#Ajout de 4 colonnes

Il y a bien 585 dossiers.
Tout est complet pour les 585 individus

In [11]:
train.head(10)

Il n'y a pas forcément le même nombre d'images dans tous les sous-dossiers en fonction des individus.
Dépend du type de plan (coronal, axial et sagital)
Dans tous les sous dossiers il y a une majorité de plan de type "axial"

In [12]:
allsame = [train["FLAIR_count"].iloc[s] ==   train["T1w_count"].iloc[s] ==train["T1wCE_count"].iloc[s] ==train["T2w_count"].iloc[s] 
          for s in range(len(train))]

Le programme ci-dessus permet de déterminer s'il y a le même nombre d'image dans les 4 coupes pour chaque individu.
Création d'une variable pour stocker cette information

In [14]:
train["allsame"] = allsame #Création de la variable 
train.head(10)

In [16]:
round(train["allsame"].sum()/len(train) * 100,2)

Commentaire : 10.77 % des individus ont le même nombre d'image dans chaque type de coupe

## EDA : Exemple 

In [18]:
row_ID = 64
train["BraTS21ID"].iloc[row_ID] #A la ligne 64 on retrouve l'individu ayant le numéro id 100

On prend l'exemple de l'individu 100 car il y a la présence de MGMT et il y a le même nombre d'image pour chaque coupe. Donc plus simple à traiter

In [19]:
temp_folder = train["path"].iloc[row_ID]